<a href="https://colab.research.google.com/github/johntiger1/hugging-face-generation/blob/master/Copy_of_t5_trivia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<a href="https://colab.research.google.com/github/google-research/text-to-text-transfer-transformer/blob/master/notebooks/t5-trivia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 The T5 Authors

Licensed under the Apache License, Version 2.0 (the "License");

In [1]:
# Copyright 2019 The T5 Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Fine-Tuning the Text-To-Text Transfer Transformer (T5) for Closed-Book Question Answering
## _Or: What does T5 know?_

*The following tutorial guides you through the process of fine-tuning a pre-trained T5 model, evaluating its accuracy, and using it for prediction,
all on a free Google Cloud TPU <a href="https://colab.research.google.com/github/google-research/text-to-text-transfer-transformer/blob/master/notebooks/t5-trivia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>.*

### Background

T5 was introduced in the paper [_Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer_](https://arxiv.org/abs/1910.10683). In that paper, we provided a comprehensive picture of how we pre-trained a standard text-to-text Transformer model on a large text corpus, achieving state-of-the-art results on many NLP tasks after fine-tuning.

We pre-trained T5 on a mixture of supervised and unsupervised tasks with the majoriy of data coming from an unlabeled dataset we developed called [C4](https://www.tensorflow.org/datasets/catalog/c4). C4 is based on a massive scrape of the web produced by [Common Crawl](https://commoncrawl.org). Loosely speaking, pre-training on C4 ideally gives T5 an understanding of natural language in addition to general world knowledge.

### How can we assess what T5 knows?

As the name implies, T5 is a text-to-text model, which enables us to train it on arbitrary tasks involving a textual input and output. As we showed in our paper, a huge variety of NLP tasks can be cast in this format, including translation, summarization, and even classification and regression tasks.

One way to use this text-to-text framework is on reading comprehension problems, where the model is fed some context along with a question and is trained to predict the question's answer. For example, we might feed the model the text from the Wikipedia article about [Hurrican Connie](https://en.wikipedia.org/wiki/Hurricane_Connie) along with the question "On what date did Hurricane Connie occur?" and train the model to predict the answer "August 3rd, 1955".
A related task is open-domain question answering (QA) where the model is not provided with this oracle context. Typically, open-domain QA systems include a mechanism to look up information in an external knowledge source. This setting is similar to an "open-book" exam.

In this notebook, we'll be training T5 on a variant of this task which we call **closed-book question answering**. In closed-book QA, we feed the model a question *without any context or access to external knowledge* and train it to predict the answer. Since the model doesn't receive any context, the primary way it can learn to answer these questions is based on the "knowledge" it obtained during pre-training. We don't expect T5 to contain super specific information, so we will be focusing on two question-answering datasets which largely include trivia questions (i.e. facts about well-known subjects). [Similar](https://arxiv.org/abs/1909.01066) [investigations](https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf) have recently been done to test the knowledge stored by BERT and GPT-2.

T5 was not pre-trained on closed-book QA, so in this notebook we'll first create two new tasks and then use the [`t5`](https://github.com/google-research/text-to-text-transfer-transformer) library to fine-tune, evaluate, and obtain predictions from T5. In the end, T5's performance on closed-book QA can give us a sense of what kind (and how much) information T5 managed to learn during pre-training.

## State-of-the-art Results
We published a [more in-depth investigation](https://arxiv.org/abs/2002.08910) of closed-book QA with T5 where we achieved SOTA on open-domain variants of WebQuestions and TriviaQA in addition to surpisingly strong results on Natural Questions. The code in this notebook is a simplified version of those experiments but still produces good results.

For code to reproduce our best results, please see the [t5_closed_book_qa](https://github.com/google-research/google-research/tree/master/t5_closed_book_qa) repo.


### Caveats

* While we provide instructions for running on a [Cloud TPU](https://cloud.google.com/tpu/) via Colab for free, a [Google Cloud Storage (GCS)](http://console.cloud.google.com/storage) bucket is required for storing model parameters and data. The [GCS free tier](https://cloud.google.com/free/) provides 5 GB of storage, which should be enough to train the `large` model and smaller but not the `3B` or `11B` parameter models. You can use part of your initial $300 credit to get more space.
* The Cloud TPU provided by Colab (a `v2-8`) does not have enough memory to fine-tune the `11B` parameter model. For this model, you will need to fine-tune inside of a GCP instance (see [README](https://github.com/google-research/text-to-text-transfer-transformer/)).


# Set Up

<h3><a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>  &nbsp;&nbsp;Train on TPU</h3>




   1. Create a Cloud Storage bucket for your data and model checkpoints at http://console.cloud.google.com/storage, and fill in the `BASE_DIR` parameter in the following form. There is a [free tier](https://cloud.google.com/free/) if you do not yet have an account.
 
   1. On the main menu, click Runtime and select **Change runtime type**. Set "TPU" as the hardware accelerator.
   1. Run the following cell and follow instructions to:
    *  Set up a Colab TPU running environment
    *   Verify that you are connected to a TPU device
    *   Upload your credentials to TPU to access your GCS bucket


In [2]:
print("Installing dependencies...")
%tensorflow_version 2.x
!pip install -q t5

import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

import t5

BASE_DIR = "gs://t5-open-qa-bucket" #@param { type: "string" }
if not BASE_DIR or BASE_DIR == "gs://":
  raise ValueError("You must enter a BASE_DIR.")
DATA_DIR = os.path.join(BASE_DIR, "data")
MODELS_DIR = os.path.join(BASE_DIR, "models")
ON_CLOUD = True


if ON_CLOUD:
  print("Setting up GCS access...")
  import tensorflow_gcs_config
  from google.colab import auth
  # Set credentials for GCS reading/writing from Colab and TPU.
  TPU_TOPOLOGY = "2x2"
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    TPU_ADDRESS = tpu.get_master()
    print('Running on TPU:', TPU_ADDRESS)
  except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
  auth.authenticate_user()
  tf.config.experimental_connect_to_host(TPU_ADDRESS)
  tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()

# Improve logging.
from contextlib import contextmanager
import logging as py_logging

if ON_CLOUD:
  tf.get_logger().propagate = False
  py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

Installing dependencies...
     |████████████████████████████████| 153kB 3.4MB/s 
     |████████████████████████████████| 3.4MB 6.0MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 675kB 35.4MB/s 
     |████████████████████████████████| 296kB 47.1MB/s 
     |████████████████████████████████| 1.1MB 45.5MB/s 
     |████████████████████████████████| 3.0MB 44.6MB/s 
     |████████████████████████████████| 890kB 50.4MB/s 
     |████████████████████████████████| 3.8MB 48.9MB/s 
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.15.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
Setting up GCS access...
Running on TPU: grpc://10.51.203.178:8470
Instructions for updating:
non-resource variables are not supported in th

# Creating new Tasks and Mixture

Two core components of the T5 library are `Task` and `Mixture` objects.

A `Task` is a dataset along with preprocessing functions and evaluation metrics. A `Mixture` is a collection of `Task` objects along with a mixing rate or a function defining how to compute a mixing rate based on the properties of the constituent `Tasks`.

For this example, we will fine-tune the model to do closed-book question answering.

### Natural Questions

[Natural Questions (NQ)](https://ai.google.com/research/NaturalQuestions) is a challenging corpus for open-domain QA. Each example includes a question along with an entire Wikipedia article that may or may not contain its answer. The goal is to produce the correct answer given this context. In our case, we will be ignoring the provided context in hopes that the model will learn to find the answers from the world knowledge it has acquired during pre-training.

Since the raw data splits are stored as JSONL files, we will first need to convert them to TSV format to make them parseable in TensorFlow. We will also take the opportunity to drop information we will not be using, remove questions with multiple answers, and to do a bit of cleaning of the text.

In [3]:
import gzip
import json

# Public directory of Natural Questions data on GCS.
NQ_JSONL_DIR = "gs://natural_questions/v1.0-simplified/"
NQ_SPLIT_FNAMES = {
    "train": "simplified-nq-train.jsonl.gz",
    "validation": "nq-dev-all.jsonl.gz"
}
nq_counts_path = os.path.join(DATA_DIR, "nq-counts.json")
nq_tsv_path = {
    "train": os.path.join(DATA_DIR, "nq-train.tsv"),
    "validation": os.path.join(DATA_DIR, "nq-validation.tsv")
}

covid_qa_tsv_path = {
    "train": os.path.join(DATA_DIR, "covid-train.tsv"),
    "validation": os.path.join(DATA_DIR, "covid-valid.tsv")

}

import pandas as pd
def csv_to_tsv(in_fname, out_fname):
  my_df = pd.load_csv(in_fname)

  

'''
We need to redo it:
formulate the COVID-19 in this format
'''
from tqdm.auto import tqdm
def nq_jsonl_to_tsv(in_fname, out_fname):

  def extract_answer(tokens, span):
    """Reconstruct answer from token span and remove extra spaces."""
    start, end = span["start_token"], span["end_token"]  
    ans = " ".join(tokens[start:end])
    # Remove incorrect spacing around punctuation.
    ans = ans.replace(" ,", ",").replace(" .", ".").replace(" %", "%")
    ans = ans.replace(" - ", "-").replace(" : ", ":").replace(" / ", "/")
    ans = ans.replace("( ", "(").replace(" )", ")")
    ans = ans.replace("`` ", "\"").replace(" ''", "\"")
    ans = ans.replace(" 's", "'s").replace("s ' ", "s' ")
    return ans

  count = 0
  with tf.io.gfile.GFile(in_fname, "rb") as infile,\
       tf.io.gfile.GFile(out_fname, "w") as outfile:
    for line in tqdm(gzip.open(infile)):
      ex = json.loads(line)
      # Remove any examples with more than one answer.
      if len(ex['annotations'][0]['short_answers']) != 1:
        continue
      # Questions in NQ do not include a question mark.
      question = ex["question_text"] + "?"
      answer_span = ex['annotations'][0]['short_answers'][0]
      # Handle the two document formats in NQ (tokens or text).
      if "document_tokens" in ex:
        tokens = [t["token"] for t in ex["document_tokens"]]
      elif "document_text" in ex:
        tokens = ex["document_text"].split(" ")
      answer = extract_answer(tokens, answer_span)
      # Write this line as <question>\t<answer>
      outfile.write("%s\t%s\n" % (question, answer)) # we just need Q TAB A pairs
      count += 1
      tf.logging.log_every_n(
          tf.logging.INFO,
          "Wrote %d examples to %s." % (count, out_fname),
          1000)
      # if count > 100 and count % 10000 == 0:
      #   print("nice, finsihed 10k examples")
      #   break
    return count

if tf.io.gfile.exists(nq_counts_path):
  # Used cached data and counts.
  tf.logging.info("Loading NQ from cache.")
  num_nq_examples = json.load(tf.io.gfile.GFile(nq_counts_path))
else:
  # Create TSVs and get counts.
  tf.logging.info("Generating NQ TSVs.")
  num_nq_examples = {}
  for split, fname in NQ_SPLIT_FNAMES.items():
    num_nq_examples[split] = nq_jsonl_to_tsv(
        os.path.join(NQ_JSONL_DIR, fname), nq_tsv_path[split])
  json.dump(num_nq_examples, tf.io.gfile.GFile(nq_counts_path, "w"))

INFO:tensorflow:Loading NQ from cache.


In [4]:
num_nq_examples

{'train': 96499, 'validation': 2338}

In [5]:
pip install gcsfs

In [6]:
DATA_DIR

'gs://t5-open-qa-bucket/data'

In [7]:
import pandas as pd


In [8]:
my_df = pd.read_csv(os.path.join(DATA_DIR,"news.csv"))


In [9]:
my_df

,source,url,question,answer,wrong_answer
0,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,What are the symptoms?,"Symptoms include fever, coughing, sore throat,...","As cases of coronavirus surge, health authorit..."
1,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,When should I get tested?,Your doctor will tell you if you need to get t...,If you have been in contact with a person with...
2,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,What's the difference between physical distanc...,"As cases of coronavirus surge, health authorit...","Remember, the vast majority of Australians are..."
3,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,How do I practice physical distancing?,If you have been in contact with a person with...,Other countries have closed schools and univer...
4,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,What's closed?,Physical distancing is the reason the Federal ...,"Symptoms include fever, coughing, sore throat,..."
...,...,...,...,...,...
478,USA Today,https://www.usatoday.com/story/news/health/202...,Is the presence of a fever always the first sy...,Symptoms of COVID-19 resemble the flu and incl...,"No, for several reasons, says Peter Hotez, dea..."
479,USA Today,https://www.usatoday.com/story/news/health/202...,How can I know if I have the coronavirus and s...,"If you have symptoms and want to get tested, t...",No. While shortness of breath is among the mos...
480,USA Today,https://www.usatoday.com/story/news/health/202...,Does testing positive mean that I have the vir...,"Yes, testing positive means that you have the ...",You should take precaution with any containers...
481,USA Today,https://www.usatoday.com/story/news/health/202...,"There has been a Hong Kong flu, a Swine flu, H...",The current outbreak is the world's first pand...,"Vaccines against pneumonia, such as pneumococc..."


In [10]:
my_df.iloc[431]

source                                                  USA Today
url             https://www.usatoday.com/story/news/health/202...
question                      What are dangers to pregnant women?
answer                                                        NaN
wrong_answer                                                  NaN
Name: 431, dtype: object

In [11]:
my_df = my_df.astype(pd.StringDtype())

In [12]:
my_df.dtypes

source          string
url             string
question        string
answer          string
wrong_answer    string
dtype: object

In [13]:

relevant_df = my_df[["question", "answer"]].dropna()

In [14]:
len(relevant_df)

479

In [15]:
# relevant_df.iloc[432,1]

In [16]:
# incredibly, some of them are floats!!
relevant_df["question"] = relevant_df["question"].map(lambda x: x.replace("\n", " "))
relevant_df["answer"] = relevant_df["answer"].map(lambda x: x.replace("\n", " "))




In [17]:
# for idx, row in relevant_df.iterrows():
#   # print(type(row["answer"]))
#   if type(row["answer"]) != str:
#     print(idx)

In [18]:

# relevant_df = my_df[["question", "answer"]]
# relevant_df["answer"] = relevant_df.loc[:,"answer"].map(lambda x: x.strip())
# relevant_df["answer"] = relevant_df["answer"].str.strip()
# relevant_df["answer"] = relevant_df["answer"].apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# map(lambda x: x)

In [19]:
my_df.dtypes

source          string
url             string
question        string
answer          string
wrong_answer    string
dtype: object

In [20]:
# relevant_df = my_df[["question", "answer"]]


In [21]:
relevant_df

,question,answer
0,What are the symptoms?,"Symptoms include fever, coughing, sore throat,..."
1,When should I get tested?,Your doctor will tell you if you need to get t...
2,What's the difference between physical distanc...,"As cases of coronavirus surge, health authorit..."
3,How do I practice physical distancing?,If you have been in contact with a person with...
4,What's closed?,Physical distancing is the reason the Federal ...
...,...,...
478,Is the presence of a fever always the first sy...,Symptoms of COVID-19 resemble the flu and incl...
479,How can I know if I have the coronavirus and s...,"If you have symptoms and want to get tested, t..."
480,Does testing positive mean that I have the vir...,"Yes, testing positive means that you have the ..."
481,"There has been a Hong Kong flu, a Swine flu, H...",The current outbreak is the world's first pand...


In [22]:
from sklearn.model_selection import train_test_split
# randchoice k indices without replacement
train_covid_df, valid_covid_df = train_test_split(relevant_df, test_size=0.05, random_state=0)

In [61]:
counts_df = train_covid_df.groupby("question").count()

In [62]:
counts_df

,answer
question,
"After recovering from coronavirus, does a person have long-term immunity to the virus?",1
"Alright, I’m in! Where do I apply?",1
"And let's start with the trends here. Who's getting sick, and how sick are they getting?",1
Are antibiotics effective in preventing or treating the COVID-19?,1
Are casual hook-ups dangerous? - May,1
...,...
Will the hospitals remain open?,1
Will the internet and couriers be suspended?,1
Will the warmer weather stop the spread?,1


In [63]:
counts_df[counts_df["answer"]>1]

,answer
question,
Are infected people able to transmit the virus to their pets and visa versa?,2
Are pregnant women at higher risk?,2
Are there any medicines or therapies that can prevent or cure COVID-19?,2
Can CoVID-19 be caught from a person who has no symptoms?,2
Can I catch COVID-19 from the feces of someone with the disease?,2
Can I exercise outside? What about going to the park?,2
Can humans become infected with the COVID-19 from an animal source?,2
Can people get checks even if they haven’t filed a tax return in recent years?,2
Can someone get the coronavirus more than once?,2


In [59]:
train_covid_df.groupby("question").count().sort_values(by="answer")

,answer
question,
"After recovering from coronavirus, does a person have long-term immunity to the virus?",1
Should I go to work?,1
Should I get on the subway? Go to the grocery store?,1
Should I be putting my money in a mattress?,1
Should I avoid public transportation?,1
...,...
Does testing positive mean that I have the virus and that I will develop symptoms?,2
Is it best to cancel routine doctor and dental appointments?,2
Is there a cure?,2


In [23]:
# valid_covid_df

In [24]:
train_covid_df.to_csv(os.path.join(DATA_DIR,"covid-train.tsv"),sep="\t",index=False, header=False)

In [25]:
valid_covid_df.to_csv(os.path.join(DATA_DIR,"covid-valid.tsv"),sep="\t", index=False,header=False)

In [26]:
NUM_COVID_EXAMPLES = {"train":len(train_covid_df), "validation":len(valid_covid_df)}

In [27]:
NUM_COVID_EXAMPLES

{'train': 455, 'validation': 24}

In [28]:
# with open(os.path.join(DATA_DIR,"abc.txt"), "w") as file:
#   file.write("hello\n")

In [29]:
with tf.io.gfile.GFile(os.path.join(DATA_DIR,"abc.txt"), "w") as file:
  file.write("hello\n")


In [30]:
# with tf.io.gfile.GFile(covid_qa_tsv_path["train"], "r") as file:
#   for line in file:
#     print(line)

Next, we define a function to load the TSV data as a `tf.data.Dataset` in TensorFlow.

In [31]:
def nq_dataset_fn(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path[split]) #add in the specific examples you want here as well 
  # Split each "<question>\t<answer>" example into (question, answer) tuple.
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["", ""],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  # Map each tuple to a {"question": ... "answer": ...} dict.
  ds = ds.map(lambda *ex: dict(zip(["question", "answer"], ex)))
  return ds

print("A few raw validation examples...")
for ex in tfds.as_numpy(nq_dataset_fn("validation").take(1)):
  # print(ex)
  print(len(ex))
  # print(ex["question"])
  # print(ex["answer"])

A few raw validation examples...
2


In [32]:
def covid_dataset_fn(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(covid_qa_tsv_path[split]) #add in the specific examples you want here as well 
  # Split each "<question>\t<answer>" example into (question, answer) tuple.
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["", ""],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  # Map each tuple to a {"question": ... "answer": ...} dict.
  ds = ds.map(lambda *ex: dict(zip(["question", "answer"], ex)))
  
  return ds

print("A few raw validation examples...")
for ex in tfds.as_numpy(covid_dataset_fn("train").take(5)):
  # print(ex)
  print(len(ex))
  # print(ex["question"])
  # print(ex["answer"])


A few raw validation examples...
2
2
2
2
2


In [33]:
test_ds = covid_dataset_fn("train")

In [34]:
# # tf.compat.v1.enable_eager_execution()

# with tf.session() as sess:
#   for ex in test_ds:
#     print(ex)

Now, we write a preprocess function to convert the examples in the `tf.data.Dataset` into a text-to-text format, with both `inputs` and `targets` fields. The preprocessor also normalizes the text by lowercasing it and removing quotes since the answers are sometimes formatted in odd ways. Finally, we prepend 'trivia question:' to the inputs so that the model knows what task it's trying to solve.

In [35]:
def trivia_preprocessor(ds):
  def normalize_text(text):
    """Lowercase and remove quotes from a TensorFlow string."""
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text,"'(.*)'", r"\1")
    return text

  def to_inputs_and_targets(ex):
    """Map {"question": ..., "answer": ...}->{"inputs": ..., "targets": ...}."""
    return {
        "inputs":
             tf.strings.join(
                 ["trivia question: ", normalize_text(ex["question"])]),
        "targets": normalize_text(ex["answer"])
    }
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

Finally, we put everything together to create a `Task`.

In [36]:
t5.data.TaskRegistry.add(
    "nq_context_free",
    # Supply a function which returns a tf.data.Dataset.
    dataset_fn=nq_dataset_fn,
    splits=["train", "validation"],
    # Supply a function which preprocesses text from the tf.data.Dataset.
    text_preprocessor=[trivia_preprocessor],
    # Lowercase targets before computing metrics.
    postprocess_fn=t5.data.postprocessors.lower_text, 
    # We'll use accuracy as our evaluation metric.
    metric_fns=[t5.evaluation.metrics.accuracy],
    # Not required, but helps for mixing and auto-caching.
    num_input_examples=num_nq_examples
)

Let's look at a few pre-processed examples from the validation set. Note they contain both the tokenized (integer) and plain-text inputs and targets.


In [37]:
nq_task = t5.data.TaskRegistry.get("nq_context_free")
ds = nq_task.get_dataset(split="validation", sequence_length={"inputs": 128, "targets": 32})
print("A few preprocessed validation examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)

A few preprocessed validation examples...
{'inputs_plaintext': b'trivia question: how tall is the actor who plays hagrid in harry potter?', 'inputs': array([22377,   822,    10,   149,  5065,    19,     8,  7556,   113,
        4805,  4244,  3496,    26,    16,     3,  3272,   651,   815,
         449,    58,     1]), 'targets_plaintext': b'6ft 1in', 'targets': array([431,  89,  17, 209,  77,   1])}
{'inputs_plaintext': b'trivia question: what is the name of manchester united stadium?', 'inputs': array([22377,   822,    10,   125,    19,     8,   564,    13,   388,
       13263, 18279, 14939,    58,     1]), 'targets_plaintext': b'old trafford', 'targets': array([  625,     3, 14793,  2590,     1])}
{'inputs_plaintext': b'trivia question: where does cleavage of the peptide bond by chymotrypsin occur?', 'inputs': array([22377,   822,    10,   213,   405,     3,  2482,     9, 11515,
          13,     8,     3, 21826,    15,  6235,    57,     3, 11971,
        8888,   651,   102,     7,  

**Note**: Instead of defining `nq_dataset_fn` and above, we also could have used the `TextLineTask` class with the `parse_tsv` preprocessor for equivalent results as follows:

```py
t5.data.TaskRegistry.add(
    "nq_context_free",
    t5.data.TextLineTask,
    split_to_filepattern=nq_tsv_path,
    text_preprocessor=[
      functools.partial(
          t5.data.preprocessors.parse_tsv,
          field_names=["question", "answer"]),
      trivia_preprocessor
    ],
    postprocess_fn=t5.data.postprocessors.lower_text, 
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples
)
```


In [38]:
t5.data.TaskRegistry.remove("covid_context_free")

In [39]:
t5.data.TaskRegistry.add(
    "covid_context_free",
    # Supply a function which returns a tf.data.Dataset.
    dataset_fn=covid_dataset_fn,
    splits=["train", "validation"],
    # Supply a function which preprocesses text from the tf.data.Dataset.
    text_preprocessor=[trivia_preprocessor],
    # Lowercase targets before computing metrics.
    postprocess_fn=t5.data.postprocessors.lower_text, 
    # We'll use accuracy as our evaluation metric.
    metric_fns=[t5.evaluation.metrics.accuracy],
    # Not required, but helps for mixing and auto-caching.
    num_input_examples=NUM_COVID_EXAMPLES
)

In [40]:
covid_qa_task = t5.data.TaskRegistry.get("covid_context_free")
covid_ds = covid_qa_task.get_dataset(split="validation", sequence_length={"inputs": 128, "targets": 32})
print("A few preprocessed validation examples...")
for ex in tfds.as_numpy(covid_ds.take(5)):
  print(ex)

A few preprocessed validation examples...
{'inputs_plaintext': b'trivia question: why are medical workers getting sick with or dying from coronavirus if they\xe2\x80\x99re wearing protective gear?', 'inputs': array([22377,   822,    10,   572,    33,  1035,  2765,   652,  6802,
          28,    42, 13677,    45,  4301,   106,     9, 18095,     3,
          99,    79,    22,    60,  5119, 13310,  4124,    58,     1]), 'targets_plaintext': b'many health care workers don\xe2\x80\x99t actually have enough protective gear to handle the growing influx of coronavirus patients.   some have resorted to using plastic report covers as masks. the cdc said medical providers might have to use expired masks or reuse them between multiple patients. (this shortage is why the general public should not buy medical masks and instead make cloth masks at home.)  but it\xe2\x80\x99s not just subpar protective gear that puts medical workers at risk. it\xe2\x80\x99s also the amount of virus they\xe2\x80\x99re 

## TriviaQA

A second dataset we will use is related to [TriviaQA](https://nlp.cs.washington.edu/triviaqa/). It is also intended for reading comprehension, but, once again, we will modify the task here by ignoring the provided context.

Since the dataset has been imported into [TensorFlow Datasets (TFDS)](https://www.tensorflow.org/datasets/catalog/trivia_qa), we can let it handle the data parsing for us. It will take a few minutes to download and preprocess the first time, but we'll be able to access it instantly from our data directory afterward.

In [41]:
ds = tfds.load(
    "trivia_qa/unfiltered.nocontext",
    data_dir=DATA_DIR,
    # Download data locally for preprocessing to avoid using GCS space.
    download_and_prepare_kwargs={"download_dir": "./downloads"})
print("A few raw validation examples...")
for ex in tfds.as_numpy(ds["validation"].take(2)):
  print(ex)

INFO:absl:Load dataset info from gs://t5-open-qa-bucket/data/trivia_qa/unfiltered.nocontext/1.1.0
INFO:absl:Reusing dataset trivia_qa (gs://t5-open-qa-bucket/data/trivia_qa/unfiltered.nocontext/1.1.0)
INFO:absl:Constructing tf.data.Dataset for split None, from gs://t5-open-qa-bucket/data/trivia_qa/unfiltered.nocontext/1.1.0


A few raw validation examples...
{'answer': {'aliases': array([b'Torquemada (disambiguation)', b'Torquemada'], dtype=object), 'matched_wiki_entity_name': b'', 'normalized_aliases': array([b'torquemada', b'torquemada disambiguation'], dtype=object), 'normalized_matched_wiki_entity_name': b'', 'normalized_value': b'torquemada', 'type': b'WikipediaEntity', 'value': b'Torquemada'}, 'entity_pages': {'doc_source': array([], dtype=object), 'filename': array([], dtype=object), 'title': array([], dtype=object), 'wiki_context': array([], dtype=object)}, 'question': b'In 1483, who was appointed the first grand inquisitor of the Spanish Inquisition?', 'question_id': b'qw_16011', 'question_source': b'http://www.quizwise.com/', 'search_results': {'description': array([], dtype=object), 'filename': array([], dtype=object), 'rank': array([], dtype=int32), 'search_context': array([], dtype=object), 'title': array([], dtype=object), 'url': array([], dtype=object)}}
{'answer': {'aliases': array([b'Auster

As with Natural Questions, we need to preprocess the raw examples into `inputs` and `targets` features. We can reuse the `trivia_preprocessor` above, but first we need to convert the TriviaQA examples into the correct format, ignoring the fields we don't need for our task.

We'll then define our `Task` and print out a few preprocessed examples from the validation set.

Note that we do not need to specify the splits or number of examples since that information is provided by TFDS.

In [42]:
def tiviaqa_extract_qa(ds):
  def exract_qa(ex):
    return {
        "question": ex["question"],
        "answer": ex["answer"]["value"]
    }
  return ds.map(exract_qa, num_parallel_calls=tf.data.experimental.AUTOTUNE)

t5.data.TaskRegistry.add(
    "triviaqa_context_free",
    # A TfdsTask takes in a TFDS name instead of a tf.data.Dataset function.
    t5.data.TfdsTask,
    tfds_name="trivia_qa/unfiltered.nocontext:1.1.0",
    tfds_data_dir=DATA_DIR,
    text_preprocessor=[tiviaqa_extract_qa, trivia_preprocessor],
    postprocess_fn=t5.data.postprocessors.lower_text,
    metric_fns=[t5.evaluation.metrics.accuracy]
)

# Load and print a few examples.
triviaqa_task = t5.data.TaskRegistry.get("triviaqa_context_free")
ds = triviaqa_task.get_dataset(split="validation", sequence_length={"inputs": 128, "targets": 32})
print("A few preprocessed validation examples...")
for ex in tfds.as_numpy(ds.take(3)):
  print(ex)

INFO:absl:Load dataset info from gs://t5-open-qa-bucket/data/trivia_qa/unfiltered.nocontext/1.1.0
INFO:absl:Reusing dataset trivia_qa (gs://t5-open-qa-bucket/data/trivia_qa/unfiltered.nocontext/1.1.0)
INFO:absl:Constructing tf.data.Dataset for split validation, from gs://t5-open-qa-bucket/data/trivia_qa/unfiltered.nocontext/1.1.0
INFO:absl:Load dataset info from gs://t5-open-qa-bucket/data/trivia_qa/unfiltered.nocontext/1.1.0


A few preprocessed validation examples...
{'inputs_plaintext': b'trivia question: the picts were an ancient tribe of people occupying what country in roman times?', 'inputs': array([22377,   822,    10,     8,  6686,    17,     7,   130,    46,
        4913, 14430,    13,   151,     3, 20817,    53,   125,   684,
          16,  3408,   648,    58,     1]), 'targets_plaintext': b'scotland', 'targets': array([   3,    7, 4310,   40,  232,    1])}
{'inputs_plaintext': b'trivia question: in 1973, which chicago building became the tallest in the world?', 'inputs': array([22377,   822,    10,    16, 17107,     6,    84,  8780,     9,
         839,   740,  1632,     8,  5065,   222,    16,     8,   296,
          58,     1]), 'targets_plaintext': b'sears tower (willis tower)', 'targets': array([ 142,  291,    7, 7293,   41, 8894,  159, 7293,   61,    1])}
{'inputs_plaintext': b'trivia question: which island is particularly associated with the artist paul gauguin?', 'inputs': array([22377,   8

## Dataset Mixture

We now create a `Mixture` from the above `Tasks`, which we will fine-tune on.

There are different ways to automatically set the rate (for example, based on the number of examples using `rate_num_examples`), but we will just hardcode an equal mixture for simplicity.

In [43]:
t5.data.MixtureRegistry.remove("trivia_all")
t5.data.MixtureRegistry.add(
    "trivia_all",
    ["nq_context_free", "triviaqa_context_free", "covid_context_free"],
     default_rate=1.0
)

# Transferring to new Tasks

We are now ready to fine-tune one of the pre-trained T5 models on our new mixture of closed-book QA tasks.

First, we'll instantiate a `Model` object using the model size of your choice. Note that larger models are slower to train and use but will likely achieve higher accuracy. You also may be able to increase accuracy by training longer with more `FINETUNE_STEPS` below.


## Caveats

* Due to its memory requirements, you will not be able to train the `11B` parameter model on the TPU provided by Colab. Instead, you will need to fine-tune inside of a GCP instance (see [README](https://github.com/google-research/text-to-text-transfer-transformer/)).
* Due to the checkpoint size, you will not be able use the 5GB GCS free tier for the `3B` parameter models. You will need at least 25GB of space, which you can purchase with your $300 of initial credit on GCP.
* While `large` can achieve decent results, it is recommended that you fine-tune at least the `3B` parameter model.


## Define Model

In [44]:
MODEL_SIZE = "large" #@param["small", "base", "large", "3B", "11B"]
# Public GCS path for T5 pre-trained model checkpoints
BASE_PRETRAINED_DIR = "gs://t5-data/pretrained_models"
PRETRAINED_DIR = os.path.join(BASE_PRETRAINED_DIR, MODEL_SIZE)
MODEL_DIR = os.path.join(MODELS_DIR, MODEL_SIZE)

if ON_CLOUD and MODEL_SIZE == "3B":
  tf.logging.warn(
      "The `3B` model is too large to use with the 5GB GCS free tier. "
      "Make sure you have at least 25GB on GCS before continuing."
  )
elif ON_CLOUD and MODEL_SIZE == "11B":
  raise ValueError(
      "The `11B` parameter is too large to fine-tune on the `v2-8` TPU "
      "provided by Colab. Please comment out this Error if you're running "
      "on a larger TPU."
  )

# Set parallelism and batch size to fit on v2-8 TPU (if possible).
# Limit number of checkpoints to fit within 5GB (if possible).
model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

tf.io.gfile.makedirs(MODEL_DIR)
# The models from our paper are based on the Mesh Tensorflow Transformer.
model = t5.models.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    sequence_length={"inputs": 128, "targets": 32},
    learning_rate_schedule=0.003,
    save_checkpoints_steps=5000,
    keep_checkpoint_max=keep_checkpoint_max if ON_CLOUD else None,
    iterations_per_loop=100,
)

Before we continue, let's load a [TensorBoard](https://www.tensorflow.org/tensorboard) visualizer so that we can keep monitor our progress. The page should automatically update as fine-tuning and evaluation proceed.

In [45]:
# if ON_CLOUD:
#   %reload_ext tensorboard
#   import tensorboard as tb
# tb.notebook.start("--logdir " + MODELS_DIR)

## Fine-tune

We are now ready to fine-tune our model. This will take a while (~2 hours with default settings), so please be patient! The larger the model and more `FINETUNE_STEPS` you use, the longer it will take.

Don't worry, you can always come back later and increase the number of steps, and it will automatically pick up where you left off.

In [46]:
FINETUNE_STEPS =  25000#@param {type: "integer"}

model.finetune(
    mixture_or_task_name="trivia_all",
    pretrained_model_dir=PRETRAINED_DIR,
    finetune_steps=FINETUNE_STEPS
)

INFO:tensorflow:Using config: {'_model_dir': 'gs://t5-open-qa-bucket/models/large', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.51.203.178:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.51.203.178:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.51.203.178:8470', '_evaluation_master': 'grpc://10.51.203.178:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_rep

## Expected Results [SPOILER ALERT]

Below are the expected accuracies on the Natural Question (NQ) and TriviQA validation sets for various model sizes. The full 11B model produces the exact text of the answer 34.5% and 25.1% of the time on TriviaQA and NQ, respectively. The 3B parameter model, which is the largest that can be trained with a free Cloud TPU in Colab, achieves 29.7% and 23.7%, respectively.

In reality, the model performs better than this since requiring exact match is too strict of a metric, as you’ll see in the examples below. This helps to explain why the model appears to perform better on TriviaQA than NQ, as the latter tends to include more long-form answers extracted from the context.

Please see our [paper on closed-book QA](https://tiny.cc/t5-qa) where achieved even better results.

<img src="https://storage.googleapis.com/t5-data/assets/t5_trivia_expected.png">

## Evaluate

We now evaluate on the validation sets of the tasks in our mixture. Accuracy results will be logged and added to the TensorBoard above.

In [47]:
# Use a larger batch size for evaluation, which requires less memory.
model.batch_size = train_batch_size * 4
model.eval(
    mixture_or_task_name="trivia_all",
    checkpoint_steps="all"
)

INFO:tensorflow:Using config: {'_model_dir': 'gs://t5-open-qa-bucket/models/large', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.51.203.178:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.51.203.178:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.51.203.178:8470', '_evaluation_master': 'grpc://10.51.203.178:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_rep

INFO:absl:Load dataset info from gs://t5-open-qa-bucket/data/trivia_qa/unfiltered.nocontext/1.1.0
INFO:absl:Reusing dataset trivia_qa (gs://t5-open-qa-bucket/data/trivia_qa/unfiltered.nocontext/1.1.0)
INFO:absl:Constructing tf.data.Dataset for split validation, from gs://t5-open-qa-bucket/data/trivia_qa/unfiltered.nocontext/1.1.0


INFO:tensorflow:Checkpoint path gs://t5-open-qa-bucket/models/large/model.ckpt-1010900
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Querying Tensorflow master (grpc://10.51.203.178:8470) for TPU system metadata.
INFO:tensorflow:Initializing TPU system (master: grpc://10.51.203.178:8470) to fetch topology for model parallelism. This might take a while.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 3317944243066766200)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 1827259258721163261)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 4638421454387366030)
INFO:tensorflow:*** Available D

INFO:absl:Load dataset info from gs://t5-open-qa-bucket/data/trivia_qa/unfiltered.nocontext/1.1.0
INFO:absl:Reusing dataset trivia_qa (gs://t5-open-qa-bucket/data/trivia_qa/unfiltered.nocontext/1.1.0)
INFO:absl:Constructing tf.data.Dataset for split validation, from gs://t5-open-qa-bucket/data/trivia_qa/unfiltered.nocontext/1.1.0


INFO:tensorflow:enable_2d_tiling: False
INFO:tensorflow:num_cores_per_replica: 1
INFO:tensorflow:computation_shape: [1, 1, 1]
INFO:tensorflow:num_replicas: 8
INFO:tensorflow:device_assignment.topology.device_coordinates: [[[0 0 0]
  [0 0 1]
  [1 0 0]
  [1 0 1]
  [0 1 0]
  [0 1 1]
  [1 1 0]
  [1 1 1]]]
INFO:tensorflow:device_assignment.core_assignment: [[[0 0 0]]

 [[0 0 1]]

 [[0 1 0]]

 [[0 1 1]]

 [[1 0 0]]

 [[1 0 1]]

 [[1 1 0]]

 [[1 1 1]]]
INFO:tensorflow:SimdMeshImpl init: Shape[model=8] LayoutRules{('experts', 'batch'), ('ensemble', 'ensemble'), ('d_ff', 'model'), ('batch', 'batch'), ('heads', 'model'), ('vocab', 'model')}
INFO:tensorflow:Device Assignment: <tensorflow.python.tpu.device_assignment.DeviceAssignment object at 0x7efb7326f390>
INFO:tensorflow:Create pnum_tensor
INFO:tensorflow:Variable decoder/block_000/layer_000/SelfAttention/k                  size 1048576      slice_size 131072       Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Va

...

INFO:tensorflow:SimdMeshImpl init: Shape[model=8] LayoutRules{('experts', 'batch'), ('ensemble', 'ensemble'), ('d_ff', 'model'), ('batch', 'batch'), ('heads', 'model'), ('vocab', 'model')}
INFO:tensorflow:Device Assignment: <tensorflow.python.tpu.device_assignment.DeviceAssignment object at 0x7efb7326f390>
INFO:tensorflow:Create pnum_tensor
INFO:tensorflow:Variable decoder/block_000/layer_000/SelfAttention/k                  size 1048576      slice_size 131072       Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable decoder/block_000/layer_000/SelfAttention/o                  size 1048576      slice_size 131072       Shape[heads=1024, d_model=1024]                             
INFO:tensorflow:Variable decoder/block_000/layer_000/SelfAttention/q                  size 1048576      slice_size 131072       Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable decoder/block_000/layer_000/SelfAttention/v                  size

INFO:absl:Load dataset info from gs://t5-open-qa-bucket/data/trivia_qa/unfiltered.nocontext/1.1.0
INFO:absl:Reusing dataset trivia_qa (gs://t5-open-qa-bucket/data/trivia_qa/unfiltered.nocontext/1.1.0)
INFO:absl:Constructing tf.data.Dataset for split validation, from gs://t5-open-qa-bucket/data/trivia_qa/unfiltered.nocontext/1.1.0


INFO:tensorflow:SimdMeshImpl init: Shape[model=8] LayoutRules{('experts', 'batch'), ('ensemble', 'ensemble'), ('d_ff', 'model'), ('batch', 'batch'), ('heads', 'model'), ('vocab', 'model')}
INFO:tensorflow:Device Assignment: <tensorflow.python.tpu.device_assignment.DeviceAssignment object at 0x7efb7326f390>
INFO:tensorflow:Create pnum_tensor
INFO:tensorflow:Variable decoder/block_000/layer_000/SelfAttention/k                  size 1048576      slice_size 131072       Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable decoder/block_000/layer_000/SelfAttention/o                  size 1048576      slice_size 131072       Shape[heads=1024, d_model=1024]                             
INFO:tensorflow:Variable decoder/block_000/layer_000/SelfAttention/q                  size 1048576      slice_size 131072       Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable decoder/block_000/layer_000/SelfAttention/v                  size

INFO:absl:Load dataset info from gs://t5-open-qa-bucket/data/trivia_qa/unfiltered.nocontext/1.1.0
INFO:absl:Reusing dataset trivia_qa (gs://t5-open-qa-bucket/data/trivia_qa/unfiltered.nocontext/1.1.0)
INFO:absl:Constructing tf.data.Dataset for split validation, from gs://t5-open-qa-bucket/data/trivia_qa/unfiltered.nocontext/1.1.0


INFO:tensorflow:SimdMeshImpl init: Shape[model=8] LayoutRules{('experts', 'batch'), ('ensemble', 'ensemble'), ('d_ff', 'model'), ('batch', 'batch'), ('heads', 'model'), ('vocab', 'model')}
INFO:tensorflow:Device Assignment: <tensorflow.python.tpu.device_assignment.DeviceAssignment object at 0x7efb7326f390>
INFO:tensorflow:Create pnum_tensor
INFO:tensorflow:Variable decoder/block_000/layer_000/SelfAttention/k                  size 1048576      slice_size 131072       Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable decoder/block_000/layer_000/SelfAttention/o                  size 1048576      slice_size 131072       Shape[heads=1024, d_model=1024]                             
INFO:tensorflow:Variable decoder/block_000/layer_000/SelfAttention/q                  size 1048576      slice_size 131072       Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable decoder/block_000/layer_000/SelfAttention/v                  size

In [48]:
train_covid_df[train_covid_df["question"].str.contains("start")]

,question,answer
148,How did it start?,The virus is one of the few that can be transm...
202,And let's start with the trends here. Who's ge...,"Sure. Well, we do have a preliminary snapshot ..."
29,How did coronavirus start?,The first case of coronavirus was detected in ...
65,How did coronavirus start?,It is thought the disease originated at an ani...
167,How did it start?,The virus is one of the few that can be transm...


In [ ]:
train_covid_df[train_covid_df["question"].str.contains("pool")] # ensure that we FORCE the QA pairs to disjoin from one another

Let's look at a few random predictions from the validation sets. Note that we measure accuracy based on an *exact match* of the predicted answer and the ground-truth answer. As a result, some of the answers are semantically correct but are counted wrong by the exact match score.

In [49]:
import random

def print_random_predictions(task_name, n=10):
  """Print n predictions from the validation split of a task."""
  # Grab the dataset for this task.
  ds = t5.data.TaskRegistry.get(task_name).get_dataset(
      split="validation",
      sequence_length={"inputs": 128, "targets": 128},
      shuffle=False)

  def _prediction_file_to_ckpt(path):
    """Extract the global step from a prediction filename."""
    return int(path.split("_")[-2])

  # Grab the paths of all logged predictions.
  prediction_files = tf.io.gfile.glob(
      os.path.join(
          MODEL_DIR,
          "validation_eval/%s_*_predictions" % task_name))
  # Get most recent prediction file by sorting by their step.
  latest_prediction_file = sorted(
      prediction_files, key=_prediction_file_to_ckpt)[-1]

  # Collect (inputs, targets, prediction) from the dataset and predictions file
  results = []
  with tf.io.gfile.GFile(latest_prediction_file) as preds:
    for ex, pred in zip(tfds.as_numpy(ds), preds):
      results.append((tf.compat.as_text(ex["inputs_plaintext"]),
                      tf.compat.as_text(ex["targets_plaintext"]),
                      pred.strip()))

  print("<== Random predictions for %s using checkpoint %s ==>\n" %
        (task_name, 
         _prediction_file_to_ckpt(latest_prediction_file)))

  for inp, tgt, pred in random.choices(results, k=10):
    print("Input:", inp)
    print("Target:", tgt)
    print("Prediction:", pred)
    print("Counted as Correct?", tgt == pred)
    print()

# print_random_predictions("triviaqa_context_free")
# print_random_predictions("nq_context_free")
print_random_predictions("covid_context_free")


<== Random predictions for covid_context_free using checkpoint 1025700 ==>

Input: trivia question: what makes this outbreak so different?
Target: unlike the flu, there is no known treatment or vaccine, and little is known about this particular virus so far. it seems to be more lethal than the flu, but the numbers are still uncertain. and it hits the elderly and those with underlying conditions particularly hard — not just those with respiratory diseases.
Prediction: unlike the flu, there is no known treatment or vaccine, and little is known about this particular virus so far. it seems to be more lethal than
Counted as Correct? False

Input: trivia question: do i have to apply to receive a payment?
Target: __ no. if you qualify, the internal revenue service already has your bank account information, it will transfer the money to you via direct deposit based on the recent income-tax figures it already has. treasury secretary steven mnuchin [said](https://www.nytimes.com/2020/03/25/world

# Predict on given questions

In [52]:
PARSA_QA_PATH = os.path.join(DATA_DIR, "CORD19_test_QA_results_df.csv")


In [53]:
additional_COVID_QA_df = pd.read_csv(PARSA_QA_PATH) # cloud read by the pandas

In [54]:
additional_COVID_QA_df

,question,text,answer,predicted_answer
0,What is the incubation period of the virus?,Of all 59 cases diagnosed as COVID-19 in the t...,"4 days (IQR, 2-7)",NaN
1,How large is the sample size used in COVID-19 ...,Of all 59 cases diagnosed as COVID-19 in the t...,59 cases,NaN
2,What is the incubation period of the virus?,Participants and study characteristics 157 We ...,"5.84 (99% CI: 4.83, 6.85) days",5.84 days
3,What is the incubation period of the virus?,We have provided an assessment of the rapidly ...,mean incubation period of 5.2 days,NaN
4,How large is the sample size used in COVID-19 ...,We have provided an assessment of the rapidly ...,49 cases,NaN
...,...,...,...,...
130,What is the OR for severe infection in COVID-1...,"As shown in Figure. 1, initial database search...","OR= 3.54, 95% CI 1.79-7.01, I 2 = 58.4%, Q-val...",NaN
131,What is the mortality rate for COVID-19 patien...,"Most routine scoring systems, such as SOFA, wh...","Patients with hypertension, showed a significa...",NaN
132,What is the OR for severe infection in COVID-1...,This study shows the conclusive evidence as ri...,"OR = 3.04 [2.01, 4.60]",intensive care and aggressive treatments are n...
133,What is the RR for severe infection in COVID-1...,This comprehensive meta-analysis of 30 studies...,"RR 2.45 [1.79, 3.35], p<0.001; I 2 : 45%, p=0.04",NaN


In [55]:
# extract only the questions
additional_COVID_QA_df["question"]
covid_predict_inputs_path = os.path.join(MODEL_DIR, "predict_inputs_COVID_QA.txt")
covid_predict_outputs_path = os.path.join(MODEL_DIR, "predict_outputs_COVID_QA.txt")
with tf.io.gfile.GFile(covid_predict_inputs_path, "w") as f:
  for idx, row in additional_COVID_QA_df.iterrows():
    q = row["question"]
    f.write("trivia question: %s\n" % q.lower()) #TODO: can also invoke the preprocess from earlier

model.batch_size = 32  # Min size for small model on v2-8 with parallelism 1.
model.predict(
    input_file=covid_predict_inputs_path,
    output_file=covid_predict_outputs_path,
    # Select the most probable output token at each step.
    temperature=0,
)


INFO:tensorflow:Using config: {'_model_dir': 'gs://t5-open-qa-bucket/models/large', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.51.203.178:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.51.203.178:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.51.203.178:8470', '_evaluation_master': 'grpc://10.51.203.178:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_rep

In [ ]:
# now, we simply get the predict outputs from the list of paths we provided

## Predict

Now that we have fine-tuned the model, we can feed T5 arbitrary questions and have it predict the answers!

There is a significant amount of overhead in initializing the model so this may take a few minutes to run each time even though the prediction itself is quite fast.


To avoid this overhead, you might consider exporting a `SavedModel` and running it on [Cloud ML Engine](https://cloud.google.com/ml-engine/).



In [50]:
question_1 = "What is known about COVID-19 therapeutics?" #@param {type:"string"}
question_2 = "What is the most populous country in the world?" #@param {type:"string"}
question_3 = "Who are the 4 members of The Beatles?" #@param {type:"string"}
question_4 = "How many teeth do humans have?" #@param {type:"string"}

questions = [question_1, question_2, question_3, question_4]

now = time.time()
# Write out the supplied questions to text files.
predict_inputs_path = os.path.join(MODEL_DIR, "predict_inputs_%d.txt" % now)
predict_outputs_path = os.path.join(MODEL_DIR, "predict_outputs_%d.txt" % now)
# Manually apply preprocessing by prepending "triviaqa question:".
with tf.io.gfile.GFile(predict_inputs_path, "w") as f:
  for q in questions:
    f.write("trivia question: %s\n" % q.lower())

# Ignore any logging so that we only see the model's answers to the questions.
# with tf_verbosity_level('ERROR'):
model.batch_size = 8  # Min size for small model on v2-8 with parallelism 1.
model.predict(
    input_file=predict_inputs_path,
    output_file=predict_outputs_path,
    # Select the most probable output token at each step.
    temperature=0,
)

# The output filename will have the checkpoint appended so we glob to get 
# the latest.
prediction_files = sorted(tf.io.gfile.glob(predict_outputs_path + "*"))
print("\nPredictions using checkpoint %s:\n" % prediction_files[-1].split("-")[-1])
with tf.io.gfile.GFile(prediction_files[-1]) as f:
  for q, a in zip(questions, f):
    if q:
      print("Q: " + q)
      print("A: " + a)
      print()

INFO:tensorflow:Using config: {'_model_dir': 'gs://t5-open-qa-bucket/models/large', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.51.203.178:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.51.203.178:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.51.203.178:8470', '_evaluation_master': 'grpc://10.51.203.178:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_rep

KeyboardInterrupt: ignored

# Export Model for Serving

As mentioned in the previous section, exporting a [`SavedModel`](https://www.tensorflow.org/guide/saved_model) can be useful for improving performance during inference or allowing your model to be deployed on a variety of platforms (e.g., TFLite, TensorFlow.js, TensorFlow Serving, or TensorFlow Hub).

**Note:** we currently only support exporting a SavedModel that runs on both CPU and GPU, not TPU.

## Export SavedModel

We first export the SavedModel. We set a batch size of 1 for simplicity, but it may be more efficient to use a larger batch size if you want to handle multiple requests per call.

For 3B and 11B models the export will take approximately 30-45 minutes.

In [ ]:
export_dir = os.path.join(MODEL_DIR, "export")

model.batch_size = 1 # make one prediction per call
saved_model_path = model.export(
    export_dir,
    checkpoint_step=-1,  # use most recent
    beam_size=1,  # no beam search
    temperature=1.0,  # sample according to predicted distribution
)
print("Model saved to:", saved_model_path)

## Load SavedModel

One way to test our model is to load it either in eager mode or a TF 1.x session so that we can repeatedly predict from the model without the overhead of loading the graph and weights each time.

We pay the overhead once here, but it shouldn't take more than a few minutes.


### Optional: Switch to GPU Runtime

Changing the runtime type to GPU in the `Runtime` menu above before loading the SavedModel will speed up inference by using the GPU instead of CPU.



In [ ]:
#@title Optional: Run this cell to re-initialize if you switched to GPU runtime.
%tensorflow_version 2.x
!pip install tensorflow-text
from google.colab import auth
auth.authenticate_user()

In [ ]:
import tensorflow as tf
import tensorflow_text  # Required to run exported model.

def load_predict_fn(model_path):
  if tf.executing_eagerly():
    print("Loading SavedModel in eager mode.")
    imported = tf.saved_model.load(model_path, ["serve"])
    return lambda x: imported.signatures['serving_default'](tf.constant(x))['outputs'].numpy()
  else:
    print("Loading SavedModel in tf 1.x graph mode.")
    tf.compat.v1.reset_default_graph()
    sess = tf.compat.v1.Session()
    meta_graph_def = tf.compat.v1.saved_model.load(sess, ["serve"], model_path)
    signature_def = meta_graph_def.signature_def["serving_default"]
    return lambda x: sess.run(
        fetches=signature_def.outputs["outputs"].name, 
        feed_dict={signature_def.inputs["input"].name: x}
    )

predict_fn = load_predict_fn(saved_model_path)

## Predict

We can now call the predict method with different inputs each time and relatively quickly get results.

In [ ]:
def answer(question):
  return predict_fn([question])[0].decode('utf-8')

for question in ["trivia question: where is the google headquarters?",
                 "trivia question: what is the most populous country in the world?",
                 "trivia question: who are the 4 members of the beatles?",
                 "trivia question: how many teeth do humans have?"]:
    print(answer(question))

## Deploy SavedModel

You can now deploy your SavedModel for serving (e.g., with [TensorFlow Serving](https://www.tensorflow.org/tfx/tutorials/serving/rest_simple)).